In [1]:
from IPython.core.display import HTML

HTML('''
<style>
.output_wrapper, .output {
    height: auto !important;
    max-height: none !important;
}
</style>
''')

In [ ]:
    import sys
    import os
    ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
    sys.path.append(ROOT)    
    from env import BalatroEnv, PARAM1_LENGTH, PARAM2_LENGTH, ActionType
    import json
    import torch
    from tensordict import TensorDict
    from IPython.display import display

    file = "../runs/replay_1750891294674274299.jsonl"

    # extract seed from filename
    basename = os.path.basename(file)
    parts = basename.replace('.jsonl','').split('_')
    # expected format: replay_{seed/timestamp}.jsonl
    seed = int(parts[1])

    env = BalatroEnv(worker_id=0, seed=seed, generate_replay=False)

    td = env.reset()
    print("Starting replay, initial obs:", td["observation"])

    with open(file) as f:
        for line in f:
            record = json.loads(line)
            # build tensordict for action
            action = {
                "action_type": torch.tensor(record["action_type"]),
                "param1": torch.tensor(record["param1"]),
                "param2": torch.tensor(record["param2"]),
            }
            td = env.step(TensorDict(action, batch_size=[]))["next"]
            # improve logging
            record["action_type"] = ActionType(record["action_type"])
            print("Step:", record, "Reward:", td["reward"].item(), "Total reward:", env.total_reward, "Done:", td["done"].item())
            os.chdir("..")
            display(env.run)
            os.chdir("ai")
            if td["done"]:
                break